## Data prep for retrieving beliefs for dialogs

**Goal:** Create a dataset to match dialogs with (possibly) relevant facts  
  
**Method:**
- [x] Use stacked_samsum as training dataset
- [x] Prepare datasets
    - [x] remove unnecessary columns
    - [x] expand the stacked dataset
    - [x] truncate on the right to create dangling examples
    - [x] augment dialog using openai to make longer

### Constants

In [2]:
model_name = "BAAI/bge-small-en-v1.5"
max_len = 512
next_concept_sep = "\n[NEXT_CONCEPT]\n"
training_input_file = "./data/train-soft.jsonl"
eval_input_file = "./data/eval.jsonl"
training_hn_file = "./data/train.jsonl"
eval_size = 12_500
seed = 42
query_prefix = "Represent this sentence for searching relevant passages: "
hf_repo_name = "julep-ai/dfe-stacked_samsum"

### Imports and utils

In [3]:
from functools import partial
import os
import random
import time

from datasets import load_dataset, load_from_disk
from FlagEmbedding import FlagModel
from FlagEmbedding.baai_general_embedding.finetune.hn_mine import find_knn_neg
from huggingface_hub import HfApi
import jsonlines as jsonl
import langchain
from langchain.cache import SQLiteCache
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from math import ceil
from numpy import cumsum, dot
from numpy.linalg import norm
from tqdm.auto import tqdm
from transformers import AutoTokenizer

#### Tokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

#### LLM

In [5]:
langchain.llm_cache = SQLiteCache(database_path=".langchain.db")
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.7)

In [6]:
prompt_template = PromptTemplate.from_template(
"""\
You are a dialog writer. Given a dialog continue it for {n} more turns in the same style as the original speakers. You can be creative in coming up with the next turns as long as you make sure that the new dialog is consistent with the previous messages.

### Example Dialog

Ken: Hi, how are you?
Ang: Just peachy! You?
Ken: I'm okay...
Ang: Just okay? What's wrong?
Ken: Just stressed; work stuff, fighting with Brad, too much going on at mom's.
Ang: Hang in there, it will get better!
Ken: I know, but it's a lot.
Ang: Can I do anything to help?
Ken: You are! Listening to me vent! LOL!
Ang: Are you at least doing anything fun this weekend?
Ken: Show Saturday night, then seeing the grandkids on Sunday at the zoo.

### Continuation

Ang: Sounds great! That will cheer you up!
Ken: Gotta run, work calls. Love you!
Ang: Love you too! Have a fantastic day!
Ken: You too!

### Input Dialog

{input_dialog}

### Continuation
"""
)

def gen_continuation(input_dialog, n=4):
    wait = round(random.uniform(0.3, 1.2), 3)
    time.sleep(wait)

    prompt = prompt_template.format(n=n, input_dialog=input_dialog)
    continuation = llm(prompt).strip()
    
    return continuation

#### Dataset load

In [7]:
# Get everything, we'll split it later
dataset = load_dataset(
    "stacked-summaries/stacked-samsum-1024", 
)


# Remove unnecessary columns
dataset = dataset.remove_columns(['chapter_length', 'summary_length', 'is_stacked',])

# Remove empty/null dialogs
dataset = dataset.filter(
    lambda row: row["dialogue"]
)

# Convert windows-style line endings to unix-style
dataset = dataset.map(
    lambda row: dict(dialogue=row["dialogue"].replace("\r\n", '\n'))
)

#### Dataset prep

In [8]:
def count_tokens(row):
    """Count tokens using the tokenizer"""

    dialogue = row["dialogue"]
    tokens = tokenizer.encode(dialogue, add_special_tokens=False)

    return dict(token_count=len(tokens))

In [9]:
# Add token count to every row in dataset
dataset = dataset.map(count_tokens)

In [10]:
def offset_left(
    dialogue: str,
    split_offset=0,
    splits=1,
    max_len=max_len,
):
    # Split dialog lines
    lines = dialogue.split("\n")

    # Count tokens per line
    toks_by_line = [
        len(tokenizer.encode(line, add_special_tokens=False))
        for line in lines
    ]

    # Cumulative sum of tokens per line
    cum_toks_by_line = cumsum(toks_by_line)

    # Total no. of tokens
    total_tokens = sum(toks_by_line)

    # Return as is if total tokens is less than max len of model
    if total_tokens <= max_len:
        return dialogue

    # Calculate step size
    step_size = ceil(total_tokens / (splits * 2))

    # Calculate left index
    left_index = 0
    for cum_toks in cum_toks_by_line:
        if cum_toks > (split_offset * step_size):
            break
            
        left_index += 1

    # Calculate right index
    right_index = 0
    for last_cum_toks in cum_toks_by_line[::-1]:
        if last_cum_toks < max_len:
            break
            
        right_index -= 1

    # Calc final section
    if right_index == 0:
        lines = lines[left_index:]
    else:
        lines = lines[left_index:right_index]

    return "\n".join(lines)

In [11]:
def truncate_lines(dialog, num=3, min=5):
    """
    Split dialog into lines and then drop the last `num` lines,
    making sure there are at least `min` lines remaining.
    """
    
    lines = dialog.split("\n")

    # If too short, return as is
    if len(lines) - num < min:
        return dialog

    if num > 0:
        return "\n".join(lines[:-num])
    else:
        return "\n".join(lines[-num:])


In [12]:
def expand_stacked(rows):
    """Expand stacked samsum dataset by splitting concepts in every summary per dialog"""
    
    # Get fields by batch
    dialogues = rows["dialogue"]
    summaries = rows["summary"]

    # Containers for final results
    is_augmented = []
    is_truncated = []
    final_dialogues = []
    final_summaries = []

    # Process every dialog and summary
    for dialogue, summary in tqdm(zip(dialogues, summaries)):
        # Split the summary by the NEXT_CONCEPT separator from the dataset
        ss = summary.split(next_concept_sep)

        # Split different conversations within the sample
        #   offset on the left to try to match relevance
        dd = [
            offset_left(d, split_offset=1) for d in dialogue.split("\n\n")
        ]

        is_truncated += [False] * len(dd)
        is_augmented += [False] * len(dd)
        final_dialogues += dd
        final_summaries += ss

        # ---
        # Now truncate and add
        truncated = [truncate_lines(d) for d in dd]

        is_augmented += [False] * len(dd)
        is_truncated += [t != d for t, d in zip(truncated, dd)]
        final_dialogues += truncated
        final_summaries += ss

        # ---
        # Now augment and add

        augmented = [
            truncate_lines(d + gen_continuation(d), num=-4)
            for d in dd
        ]
        
        is_truncated += [False] * len(dd)
        is_augmented += [True] * len(dd)
        final_dialogues += augmented
        final_summaries += ss

    return dict(
        dialogue=final_dialogues,
        summary=final_summaries,
        is_truncated=is_truncated,
        token_count=[None]*len(final_summaries),
    )

In [13]:
# Use batched mode to be able to expand the size of the dataset
dataset = dataset.map(expand_stacked, batch_size=10, batched=True, num_proc=75)
dataset = dataset.remove_columns(["token_count"])

Parameter 'function'=<function expand_stacked at 0x7f0a3a68eef0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map (num_proc=75):   0%|          | 0/29441 [00:00<?, ? examples/s]

Map (num_proc=75):   0%|          | 0/1633 [00:00<?, ? examples/s]

Map (num_proc=75):   0%|          | 0/1637 [00:00<?, ? examples/s]

In [14]:
dataset.push_to_hub(hf_repo_name)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/339 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/19 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

### Prepare dataset for finetuning
[Docs](https://github.com/FlagOpen/FlagEmbedding/tree/master/examples/finetune)

Format:
```json
{"query": str, "pos": List[str], "neg":List[str]}
```

Keys:
- query: belief
- pos: list of matching conversations
- neg: list of random conversations from dataset

In [4]:
dataset = load_dataset(hf_repo_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/338127 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19131 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/18381 [00:00<?, ? examples/s]

In [5]:
def pick_random(dataset, split="train", far_from=0):
    ds = dataset[split]
    ds_len = len(ds)
    mid = ds_len // 2
    which_half = far_from // mid
    
    start = (1 - which_half) * mid
    end = ds_len - which_half * mid
    idx = random.randrange(start, end)
    
    return ds[idx]

In [6]:
with jsonl.open(training_input_file, mode='w') as writer:
    for i, row in enumerate(tqdm(dataset["train"], total=len(dataset["train"]))):
        query = row["summary"]
        pos = [row["dialogue"]]
    
        neg = [
            pick_random(dataset, split="train", far_from=i)["dialogue"]
            for _ in range(3)
        ]
        
        writer.write(dict(query=query, pos=pos, neg=neg))

  0%|          | 0/338127 [00:00<?, ?it/s]

In [7]:
with jsonl.open(eval_input_file, mode='w') as writer:
    for i, row in enumerate(tqdm(dataset["validation"], total=eval_size)):
        if i > eval_size:
            break

        query = row["summary"]
        pos = [row["dialogue"]]
    
        neg = [
            pick_random(dataset, split="validation", far_from=i)["dialogue"]
            for _ in range(3)
        ]
        
        writer.write(dict(query=query, pos=pos, neg=neg))

  0%|          | 0/12500 [00:00<?, ?it/s]

### Mine hard negatives

In [9]:
model = FlagModel(
    model_name,
    query_instruction_for_retrieval=query_prefix,
)

In [10]:
find_knn_neg(
    model,
    input_file=training_input_file,
    candidate_pool=None,
    output_file=training_hn_file,
    sample_range=list(range(2, 200)),
    negative_number=10,
    use_gpu=True,
)

inferencing embedding for corpus (number=37361)--------------


Inference Embeddings: 100%|██████████| 146/146 [00:37<00:00,  3.87it/s]


inferencing embedding for queries (number=338127)--------------


Inference Embeddings: 100%|██████████| 1321/1321 [00:52<00:00, 25.34it/s]


create index and search------------------


Batches: 100%|██████████| 5284/5284 [00:07<00:00, 740.63it/s]


### Add processed files to hf dataset

In [11]:
hf_api = HfApi()

for path in [
    training_input_file,
    eval_input_file,
    training_hn_file,
]:
    hf_api.upload_file(
        path_or_fileobj=path,
        path_in_repo=path.split('/')[-1],
        repo_id=hf_repo_name,
        repo_type="dataset",
    )


train.jsonl:   0%|          | 0.00/2.42G [00:00<?, ?B/s]